In [1]:
"""
ML pipeline
"""
from scripts import logistic_regression, sigmoid
import numpy as np

# Import data
from helpers import *
DATA_TRAIN_PATH = 'data/train.csv' # TODO: download train data and supply path here 
y, tX, ids = load_csv_data(DATA_TRAIN_PATH)


In [2]:
y_shifted, tX_norm = pre_process_data(y, tX)

In [7]:
# Run logisitic regression algorithm
N, D = tX.shape
max_iters = 500
batch_size = N/100
gamma=0.00035938136638046257

loss, w = logistic_regression(y_shifted, tX_norm, gamma, max_iters, batch_size)
print(loss, w)


C:\Users\Fearnley\Documents\EPFL\Machine learning\ML_pro_1\pro1\scripts.py:162: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  y_batch = np.zeros((batch_size,1))
C:\Users\Fearnley\Documents\EPFL\Machine learning\ML_pro_1\pro1\helpers.py:47: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  yield shuffled_y[start_index:end_index], shuffled_tx[start_index:end_index]


[ 1355.81046165] [[-0.14811884]
 [-0.18089259]
 [-0.32998163]
 [-0.36853207]
 [ 0.90762942]
 [-0.34420668]
 [ 1.11060347]
 [ 0.10000989]
 [ 0.90127521]
 [-0.17258339]
 [ 0.07507517]
 [-0.64302249]
 [ 0.2843205 ]
 [ 0.4042932 ]
 [ 0.34154496]
 [ 0.00308468]
 [-0.02282005]
 [ 0.61624712]
 [-0.01503443]
 [-0.00311258]
 [ 0.07915558]
 [ 0.00471699]
 [-0.32884507]
 [-0.17242641]
 [-0.55296511]
 [-0.00241643]
 [-0.00387323]
 [ 0.04143153]
 [ 0.004789  ]
 [-0.01287548]
 [-0.16717194]]


In [6]:
# Output predictions

#Load test data
DATA_TEST_PATH = 'data/test.csv' # TODO: download train data and supply path here 
OUTPUT_PATH = 'data/output2.csv' # TODO: fill in desired name of output file for submission
_, tX_test, ids_test = load_csv_data(DATA_TEST_PATH)

# Process test data
_, tX_test = pre_process_data(_,tX_test)


y_est = sigmoid(np.dot(tX_test,w))
y_label = [-1 if i<0.5 else 1 for i in y_est]
create_csv_submission(ids_test, y_label, OUTPUT_PATH)